In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch

### CodeLlama 7B

In [3]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf", cache_dir="./models")
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf", cache_dir="./models")

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.70s/it]


In [4]:
from transformers import pipeline
import torch

code_continuator = pipeline('text-generation',
                            model=model,
                            tokenizer=tokenizer,
                            torch_dtype=torch.float16,
                            config={"temperature": 0.2},
                            device=0
                            )

In [18]:
def continue_code(prompt):
    sequences = code_continuator(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=128,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        
def generate_code(prompt):
    prompt = "requirements: " + prompt + "\n code: "
    
    sequences = code_continuator(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=128,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        
def generate_code2(prompt):
    prompt = "#" + prompt
    
    sequences = code_continuator(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=128,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        
def summarize_code(prompt):
    prompt = "code: " + prompt + "\n summarization: "
    
    sequences = code_continuator(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=128,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [19]:
continue_code("def primes(n):")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: def primes(n):
    """
    Returns a list of the first n prime numbers.
    """
    primes = []
    for i in range(2, n + 1):
        if is_prime(i):
            primes.append(i)
    return primes


def is_prime(n):
    """
    Returns True if n is prime, False otherwise.
    """
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range


In [7]:
generate_code("gives a python function that generate prime numbers under 100")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: requirements: gives a python function that generate prime numbers under 100/n code: 

```python
def generate_prime_numbers(n):
    def is_prime(num):
        for i in range(2, num):
            if num % i == 0:


In [17]:
generate_code2("gives a python function that generate prime numbers under 100")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: #gives a python function that generate prime numbers under 1000

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False


In [ ]:
summarize_code("""
               
""")

### GPT2

In [3]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir="./models")
model = AutoModelForCausalLM.from_pretrained("gpt2", cache_dir="./models")

In [4]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [5]:
generator("requirements: give me a python function that prints all prime numbers under 100 \n code:", max_length=64, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'requirements: give me a python function that prints all prime numbers under 100 \n code:'}]

### Classification GPT2 Demo

In [3]:
import torch
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer, AdamW
from torch.nn import Linear, Sigmoid, BCEWithLogitsLoss
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/red/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/red/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model_name = 'gpt2'
local_cache_dir = "./models"

tokenizer = GPT2Tokenizer.from_pretrained(model_name, cache_dir=local_cache_dir)
config = GPT2Config.from_pretrained(model_name, cache_dir=local_cache_dir)
model = GPT2Model.from_pretrained(model_name, config=config, cache_dir=local_cache_dir)

In [9]:
class GPT2ForClassification(torch.nn.Module):
    def __init__(self, model, num_labels=2):
        super(GPT2ForClassification, self).__init__()
        self.gpt2 = model
        self.classifier = Linear(config.n_embd, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state[:, -1, :]
        logits = self.classifier(hidden_states)
        return logits

In [10]:
classification_model = GPT2ForClassification(model)

input_text = "This is a sample text for classification."
inputs = tokenizer(input_text, return_tensors="pt")
logits = classification_model(**inputs)

sigmoid = Sigmoid()
probs = sigmoid(logits)

print(probs)

tensor([[0.0473, 0.0005]], grad_fn=<SigmoidBackward0>)


In [ ]:
df = pd.read_csv("training_data.csv")
train_df, val_df = train_test_split(df, test_size=0.1)

class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        label = torch.tensor(int(self.labels[idx]), dtype=torch.float)
        return input_ids, attention_mask, label

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Create the dataset and dataloader
train_dataset = TextDataset(tokenizer, train_df['code'].tolist(), train_df['need completion or not'].tolist())
val_dataset = TextDataset(tokenizer, val_df['code'].tolist(), val_df['need completion or not'].tolist())
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Model, optimizer, and loss function
model = GPT2ForClassification(GPT2Model.from_pretrained('gpt2'))
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = BCEWithLogitsLoss()

# Training loop
model.train()
for epoch in range(4):  # Assuming 4 epochs for simplicity
    for input_ids, attention_mask, labels in train_loader:
        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(logits.view(-1), labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Save the model
torch.save(model.state_dict(), "gpt2_classification_model.pth")


### Stable Code 3B

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

local_cache_dir = "./models"

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-3b", cache_dir=local_cache_dir)
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stable-code-3b",
  torch_dtype="auto",
  cache_dir=local_cache_dir
)
model.cuda()
inputs = tokenizer("import torch\nimport torch.nn as nn", return_tensors="pt").to(model.device)
tokens = model.generate(
  **inputs,
  max_new_tokens=48,
  temperature=0.2,
  do_sample=True,
)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))


/Users/red/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/red/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


KeyboardInterrupt: 